In [18]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats import mode
from scipy.stats.distributions import randint, uniform, expon, reciprocal
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
import re

In [19]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

In [20]:
X_train = train.drop('Survived', axis=1)
y_train = train['Survived'].ravel()

In [21]:
def Get_Title(word):
    atr = re.search('(\w+)\.', word)
    if not atr:
        return 'No'
    return atr.group()

In [22]:
class Transform_Ticket(TransformerMixin, BaseEstimator):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        Xt['Ticket_Num'] = X['Ticket'].apply(lambda x: x.split(' ')[-1])
        return Xt

In [23]:
class Transform_Name_to_Title(TransformerMixin, BaseEstimator):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        Xt['Title'] = X['Name'].apply(Get_Title)
        return Xt

In [24]:
class Transform_Name_to_Surname(TransformerMixin, BaseEstimator):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        Xt['Surname'] = X['Name'].apply(lambda x: x.split(' ')[0])
        return Xt

In [25]:
class Surname_to_Feature(TransformerMixin, BaseEstimator):
    def __init__(self, high=0.7, low=0.3):
        self.high = high
        self.low = low
    def fit(self, X, y=None):
        Xt = Transform_Name_to_Surname().fit_transform(X)
        Xt['Survived'] = y.copy()
        dt = Xt.groupby('Surname')['Survived'].agg([('Count', np.size), ('Prop', np.mean)]).reset_index()
        self.SurHigh = list(dt[(dt['Count'] > 1) & (dt['Prop'] > self.high)]['Surname'])
        self.SurLow = list(dt[(dt['Count'] > 1) & (dt['Prop'] < self.low)]['Surname'])
        return self
    def transform(self, X, y=None):
        Xaux = Transform_Name_to_Surname().fit_transform(X).reset_index(drop=True)
        Xt = X.copy().reset_index(drop=True)
        Xt['High'] = 0
        Xt['Low'] = 0
        for i in range(len(X)):
            sur = Xaux.loc[i,'Surname']
            if sur in self.SurHigh:
                Xt.loc[i, 'High'] = 1
            elif sur in self.SurLow:
                Xt.loc[i, 'Low'] = 1
        return Xt        

In [26]:
def write_test(df, mod, n=1):
    dft = pd.DataFrame(np.c_[df['PassengerId'],mod.predict(df)], columns=['PassengerId', 'Survived'])
    dft.to_csv('Predictions_Teste{}.csv'.format(n), index=False)

In [27]:
class Impute_Better(BaseEstimator, TransformerMixin):
    """ Impute missing data in a more effective way, groupwise by Sex and Pclass"""
    def __init__(self, col='Age', func=np.mean, mode=2):
        self.col = col
        self.func = func
        self.mode = mode
    def fit(self, X, y=None):
        if self.mode == 2:
            self.vals = X.groupby(by=['Sex', 'Pclass'])[self.col].agg(self.func)
        elif self.mode == 1:
            self.vals = X.groupby(by=['Sex'])[self.col].agg(self.func)
        else:
            self.vals = X[self.col].agg(self.func)
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        idx_na = X.loc[:,self.col].isna()
        if self.mode == 2:
            for i in range(len(self.vals)):
                Xt.loc[((X['Sex'] == self.vals.index[i][0]) & (X['Pclass'] == self.vals.index[i][1]) &
                       idx_na),self.col] = self.vals[i]
        elif self.mode == 1:
            for i in range(len(self.vals)):
                Xt.loc[((X['Sex'] == self.vals.index[i]) & idx_na),self.col] = self.vals[i]
        else:
            Xt.loc[idx_na, self.col] = self.vals
        return Xt

In [69]:
class New_Features(BaseEstimator, TransformerMixin):
    """ Add new features, in this case Parch + SibSp """
    def __init__(self, binary=False):
        self.binary = binary
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        Xt['Nrelatives'] = Xt['Parch'] + Xt['SibSp']
        if self.binary:
            Xt['Nrelatives'] = (Xt['Nrelatives'] == 0)
        return Xt

In [29]:
class Transf_Conti(BaseEstimator, TransformerMixin):
    """ Transforms a feature in categorical or startardizes it"""
    def __init__(self, n=4, q=False, conti=False):
        self.n = n
        self.q = q
        self.conti=conti
    def fit(self, X, y=None):
        if not self.conti:
            if not self.q:
                self.binst = pd.cut(X.values[:,0], bins=self.n, retbins=True, duplicates='drop')[1]
            if self.q:
                self.binst = pd.qcut(X.values[:,0], q=self.n, retbins=True, duplicates='drop')[1]
            self.higher = self.binst[-1]
            self.lower = self.binst[0]
        else:
            self.scaler_transf = StandardScaler()
            self.scaler_transf.fit(X)
        
        return self
    def transform(self, X, y=None):
        if not self.conti:
            val_ret = pd.cut(X.values[:,0], bins=self.binst, labels=False, include_lowest=True, duplicates='drop')
            val_ret[X.values[:,0] > self.higher] = self.n - 1
            val_ret[X.values[:,0] < self.lower] = 0
            return val_ret.reshape((-1, 1))
        else:
            return self.scaler_transf.transform(X)

In [30]:
class Select_Features(BaseEstimator, TransformerMixin):
    def __init__(self, select=False):
        self.select=select
    def fit(self, X, y=None):
        return self
    def transform(self, X,y=None):
        if not self.select:
            return X
        return X[:, self.select]

In [70]:
Pipe_Impute = Pipeline([
    #Impute Pipeline, the one which imputes missing data and creates new features
    ('Age_Impute', Impute_Better(col='Age', func=np.median)),
    ('Fare_Impute', Impute_Better(col='Fare', func=np.median)),
    #('Embarked_Impute', Impute_Better(col='Embarked', func=lambda x: mode(x)[0][0])),
    ('New_Features', New_Features()),
    ('Surname_to_Code', Surname_to_Feature())
])

In [71]:
Pipe_Preprocess = ColumnTransformer([
    ### Preprocessing pipeline. Encodes, standardizes
    ('Ordinal_Encode', OrdinalEncoder(), ['Sex','Pclass']), 
    ('Age_Encode', Transf_Conti(), ['Age']), 
    ('Fare_Encode', Transf_Conti(), ['Fare']), 
    ('Keep', 'passthrough', ['Parch', 'SibSp', 'Nrelatives', 'High', 'Low']), 
    #('Embarked_Encode', OneHotEncoder(), ['Embarked'])
])

In [138]:
FullPipePredict_RF = Pipeline([
    ('Impute_Pipeline', Pipe_Impute),
    ('Preprocess_Pipeline', Pipe_Preprocess),
    ('Feature_Selection', Select_Features()),
    ('Predict', RandomForestClassifier(n_jobs=-1))
])

In [160]:
param_RF = {
    'Preprocess_Pipeline__Age_Encode__conti':[1],
    'Preprocess_Pipeline__Fare_Encode__conti':[1],
    'Impute_Pipeline__Surname_to_Code__high':[0.8],
    'Impute_Pipeline__Surname_to_Code__low':[0.35],
    'Feature_Selection__select':[[0,1,2,3,6,7,8]],
    'Predict__n_estimators':[166],
    'Predict__max_leaf_nodes':[31],
}

In [161]:
grid_RF = RandomizedSearchCV(FullPipePredict_RF, param_distributions=param_RF, cv=10, n_iter=60,
                            scoring='accuracy')

In [162]:
grid_RF.fit(X_train, y_train)

C:\Users\vmrod\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 1 is smaller than n_iter=60. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('Impute_Pipeline',
                                              Pipeline(steps=[('Age_Impute',
                                                               Impute_Better(func=<function median at 0x000002E78CCE1940>)),
                                                              ('Fare_Impute',
                                                               Impute_Better(col='Fare',
                                                                             func=<function median at 0x000002E78CCE1940>)),
                                                              ('New_Features',
                                                               New_Features()),
                                                              ('Surname_to_Code',
                                                               Surname_to_Feature())])),
                                             ('Preprocess_Pipeline',
                

In [163]:
grid_RF.best_score_

0.8338576779026218

In [155]:
grid_RF.best_params_

{'Feature_Selection__select': [0, 1, 2, 3, 6, 7, 8],
 'Impute_Pipeline__Surname_to_Code__high': 0.7245766342181759,
 'Impute_Pipeline__Surname_to_Code__low': 0.365299845673262,
 'Predict__max_leaf_nodes': 31,
 'Predict__n_estimators': 183,
 'Preprocess_Pipeline__Age_Encode__conti': 1,
 'Preprocess_Pipeline__Fare_Encode__conti': 1}

In [164]:
best_RF = grid_RF.best_estimator_

In [68]:
joblib.dump(best_RF[-1], 'BestRFNames.pkl')

['BestRFNames.pkl']

In [159]:
joblib.load('BestRFNames.pkl')

RandomForestClassifier(max_leaf_nodes=31, n_estimators=166, n_jobs=-1)

In [147]:
best_RF[-1].feature_importances_

array([0.36257446, 0.08108021, 0.11569964, 0.15482718, 0.05363498,
       0.1111556 , 0.12102793])

In [165]:
write_test(test, best_RF, '_RF_Names7')